In [5]:
import logging
import pandas as pd
import os
import numpy as np

In [6]:
from retrieve_post_by_userid import RetrievePostByUserId
t = RetrievePostByUserId(sessionid="")

In [15]:
def get_post_data(data,userid): 
    post_list = [] 
    if data is not None and data["data"] is not None and data["data"]["mediaData"] is not None and data["data"]["mediaData"]["edges"] is not None: 
        edges = data["data"]["mediaData"]["edges"] 
        if edges == []: 
            return [ 
                { 
                    "id": None, 
                    "post_pk": None, 
                    "caption_text": None, 
                    "like_count": None, 
                    "taken_at":None, 
                    "username": None, 
                    "user_pk": userid, 
                    "quote_count": None, 
                    "repost_count": None, 
                    "reshare_count": None, 
                } 
            ] 
        else: 
            try: 
                for edge in edges: 
                    thread_items = edge["node"]["thread_items"] 
                    for thread_item in thread_items: 
                        post = thread_item["post"] 
                        if post["caption"] is None or post["user"] is None or post["text_post_app_info"] is None: 
                            current_post = { 
                            "id": post.get("id",""), 
                            "post_pk": post.get("pk",""), 
                            "caption_text": None if post["caption"] is None else post.get("caption",{}).get("text",""), 
                            "like_count": post.get("like_count",""), 
                            "taken_at": post.get("taken_at",""), 
                            "username": None if post["user"] is None else post.get("user",{}).get("username",""), 
                            "user_pk": post.get("user",{}).get("pk",""), 
                            "quote_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("quote_count",0), 
                            "repost_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("repost_count",0), 
                            "reshare_count": None if post["text_post_app_info"] is None else post["text_post_app_info"].get("reshare_count",0), 
                        } 
                        else: 
                            current_post = { 
                                "id": post.get("id",""), 
                                "post_pk": post.get("pk",""), 
                                "caption_text": post.get("caption",{}).get("text",""), 
                                "like_count": post.get("like_count",""), 
                                "taken_at": post.get("taken_at",""), 
                                "username": post.get("user",{}).get("username",""), 
                                "user_pk": post.get("user",{}).get("pk",""), 
                                "quote_count": post["text_post_app_info"].get("quote_count",0), 
                                "repost_count": post["text_post_app_info"].get("repost_count",0), 
                                "reshare_count": post["text_post_app_info"].get("reshare_count",0), 
                            } 
                        post_list.append(current_post) 
                return post_list 
            except Exception as e: 
                raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}") 
    if data["errors"]: 
        for error in data["errors"]: 
            if error.get("message") == "execution error" and error.get("description") == "cannot see user": 
                return [ 
                        { 
                            "id": None, 
                            "post_pk": None, 
                            "caption_text": None, 
                            "like_count": None, 
                            "taken_at":None, 
                            "username": None, 
                            "user_pk": userid, 
                            "quote_count": None, 
                            "repost_count": None, 
                            "reshare_count": None, 
                        } 
                    ] 
    return []

def get_all_post(userid: str): 
    results = [] 
    try: 
        response = t.retrieve_thread_by_userid(userid) 
        d_out = get_post_data(response,userid) 
        results.extend(d_out) 
    except Exception as e: 
        print(response) 
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}") 
    return results

In [8]:
# df = pd.read_csv("data_parte1.csv")
# df['user_pk'] = df['user_pk'].astype(str)
# unique_user_pk = df['user_pk'].unique()
# len(unique_user_pk)

In [16]:
df2 = pd.read_csv("parte_1_followers_machinelearning.csv")
df2['follower_pk'] = df2['follower_pk'].astype(str)
unique_user = df2['follower_pk'].unique()
len(unique_user)

4820

In [10]:
# unique_user = np.concatenate([unique_user_pk, unique_user_pk2])
# len(unique_user)

In [18]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "post_data_machine.csv"
processed_users = load_processed_users(csv_file)

In [19]:
len(processed_users)

816

In [20]:
import pandas as pd
i = len(processed_users)
for user in unique_user:
     if user not in processed_users:
        i+=1
        posts = get_all_post(user)
        print(user)
        final_df = pd.DataFrame(posts)
        save_to_csv(final_df, csv_file)
        print(f"Processati {i} utenti su {len(unique_user)}")

print("Processo completato!")

44779951729
Processati 817 utenti su 4820
28562060676
Processati 818 utenti su 4820
56782815109
Processati 819 utenti su 4820
273611655
Processati 820 utenti su 4820
6008461134
Processati 821 utenti su 4820
48016673169
Processati 822 utenti su 4820
5668945995
Processati 823 utenti su 4820
6443374183
Processati 824 utenti su 4820
198181538
Processati 825 utenti su 4820
771306868
Processati 826 utenti su 4820
4399903611
Processati 827 utenti su 4820
20125208
Processati 828 utenti su 4820
268927167
Processati 829 utenti su 4820
69563264223
Processati 830 utenti su 4820
5890337938
Processati 831 utenti su 4820
46599443922
Processati 832 utenti su 4820
1200754340
Processati 833 utenti su 4820
33034344237
Processati 834 utenti su 4820
6183965744
Processati 835 utenti su 4820
46758209999
Processati 836 utenti su 4820
5930337415
Processati 837 utenti su 4820
513750904
Processati 838 utenti su 4820
283321404
Processati 839 utenti su 4820
1099385602
Processati 840 utenti su 4820
9195626764
Proce

In [63]:
df1 = pd.read_csv("post_data_machine").drop_duplicates()
print(df1.shape)
df1

(260285, 10)


,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count
0,3557970465522674878_187264532,3557970465522674878,A little over 24 hours left to vote.\n\nhttps:...,0,1738363153,mlssfshn,187264532,0.0,0.0,NaN
1,3557782854629238181_187264532,3557782854629238181,"Galaxy Quest \nLots of cutting holes, playing ...",1,1738340790,mlssfshn,187264532,0.0,0.0,NaN
2,3557776470965660229_187264532,3557776470965660229,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0,1738340029,mlssfshn,187264532,0.0,0.0,NaN
3,3556000079513982405_187264532,3556000079513982405,Over due to post videos of my work for @salina...,0,1738128266,mlssfshn,187264532,0.0,0.0,NaN
4,3555273435077793954_187264532,3555273435077793954,My son has a good point.\nWhy are we calling m...,108,1738041642,mlssfshn,187264532,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
260280,3547551147648755490_69158167753,3547551147648755490,"Go-Getter: Your Destination for Style, Success...",4,1737121073,go_getter_net,69158167753,0.0,0.0,NaN
260281,3547080123266187916_69158167753,3547080123266187916,"Go-Getter: Your Destination for Style, Success...",2,1737064923,go_getter_net,69158167753,0.0,0.0,NaN
260282,3559086546466880502_49997051478,3559086546466880502,After a long time##long##time##viralreels ##vi...,0,1738496202,bankerkeshvipp_1,49997051478,0.0,0.0,NaN
260283,3559048310029913237_4922349813,3559048310029913237,My soul’s lovely 🍃evening🌚,0,1738491644,varun._.sabari,4922349813,0.0,0.0,NaN
